In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')

import time, datetime
import re
import os

import pyodbc
import urllib
from sqlalchemy import create_engine

from model_development import *

In [2]:
np.random.seed(16777216)
random.seed(16777216)

In [3]:
con = 'Driver={SQL Server};Server=;uid=;pwd=;'
ch = pyodbc.connect(con)
quoted = urllib.parse.quote_plus(con)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [4]:
n_a = 100000
n_l = 10

In [5]:
appl = pd.DataFrame({'application_id': [i for i in range(n_a)]})

In [6]:
appl['year_quarter'] = np.random.choice([20191, 20192, 20193, 20194], n_a, p=[0.25, 0.25, 0.25, 0.25])
appl['application_date'] = datetime.date(2019, 1, 1)
appl['application_date'][appl['year_quarter'] == 20191] = [datetime.date(2019, 1, 1) + (datetime.date(2019, 3, 31) - datetime.date(2019, 1, 1)) * random.random() for  i in range(n_a)]
appl['application_date'][appl['year_quarter'] == 20192] = [datetime.date(2019, 4, 1) + (datetime.date(2019, 6, 30) - datetime.date(2019, 4, 1)) * random.random() for  i in range(n_a)]
appl['application_date'][appl['year_quarter'] == 20193] = [datetime.date(2019, 7, 1) + (datetime.date(2019, 9, 30) - datetime.date(2019, 7, 1)) * random.random() for  i in range(n_a)]
appl['application_date'][appl['year_quarter'] == 20194] = [datetime.date(2019, 10, 1) + (datetime.date(2019, 12, 31) - datetime.date(2019, 10, 1)) * random.random() for  i in range(n_a)]
appl['segment'] = np.random.choice(['АК', 'ИК', 'КК', 'ПК'], n_a, p=[0.1, 0.2, 0.3, 0.4])
appl['exist'] = np.random.choice([0, 1], n_a, p=[0.6, 0.4])
appl['target'] = 0
appl['target'][appl['segment'] == 'АК'] = np.random.choice([0, 1], n_a, p=[0.98, 0.02])
appl['target'][appl['segment'] == 'ИК'] = np.random.choice([0, 1], n_a, p=[0.99, 0.01])
appl['target'][appl['segment'] == 'КК'] = np.random.choice([0, 1], n_a, p=[0.96, 0.04])
appl['target'][(appl['segment'] == 'КК') & (appl['year_quarter'] == 20193)] = np.random.choice([0, 1], n_a, p=[0.92, 0.08])
appl['target'][appl['segment'] == 'ПК'] = np.random.choice([0, 1], n_a, p=[0.95, 0.05])
appl['target'][(appl['segment'] == 'ПК') & (appl['exist'] == 1)] = np.random.choice([0, 1], n_a, p=[0.98, 0.02])

In [7]:
target0 = set(appl['application_id'][appl['target'] == 0])
target1 = set(appl['application_id'][appl['target'] == 1])
atarget0 = np.array(list(target0))
atarget1 = np.array(list(target1))

In [8]:
over = pd.DataFrame({'over_application_id': [i for i in range(n_a)]})

In [9]:
over['payload'] = [np.clip(np.random.normal(15000, 10000), 0, np.inf) for i in range(n_a)]
over['inquiries_1month'] = [np.clip(np.random.normal(4, 2), 0, np.inf) + (np.clip(np.random.normal(3, 2), 0, np.inf) if i in target1 else 0) for i in range(n_a)]
over['inquiries_3month'] = [(i + np.random.uniform(0, 1)) * (np.random.normal(2, 0.2) if idx in target1 else np.random.normal(2.2, 0.3)) for idx, i in enumerate(over['inquiries_1month'])]
over['inquiries_6month'] = [(i + np.random.uniform(0, 1)) * np.random.normal(1.8, 0.1) for i in over['inquiries_3month']]
over['inquiries_9month'] = [(i + np.random.uniform(0, 1)) * np.random.normal(2.7, 0.2) for i in over['inquiries_3month']]
over['inquiries_12month'] = [(i + np.random.uniform(0, 1)) * np.random.normal(3.5, 0.3) for i in over['inquiries_3month']]
over['inquiries_total'] = [i + (np.random.uniform(0, 1) * np.random.normal(4.5, 1)) for i in over['inquiries_12month']]
over['inquiries_1month'] = over['inquiries_1month'].astype(int)
over['inquiries_3month'] = over['inquiries_3month'].astype(int)
over['inquiries_6month'] = over['inquiries_6month'].astype(int)
over['inquiries_9month'] = over['inquiries_9month'].astype(int)
over['inquiries_12month'] = over['inquiries_12month'].astype(int)
over['inquiries_total'] = over['inquiries_total'].astype(int)

In [10]:
loan = pd.DataFrame({'loan_application_id': [i for i in range(n_a)]})
al = pd.DataFrame({'loan_id': [i for i in range(n_a * n_l)]})
al['loan_application_id'] = [np.random.choice(atarget0, 1)[0] if np.random.random() > 0.02 else np.random.choice(atarget1, 1)[0] for i in range(n_a * n_l)]

In [11]:
loan['application_date'] = appl['application_date']
loan = loan.merge(al, 'inner', 'loan_application_id')

In [12]:
la_id = np.array(loan['loan_application_id'])
t0, t1 = [], []
prev = 0
prev_d0 = datetime.timedelta()
prev_d1 = datetime.timedelta()
for i in la_id:
    if i != prev:
        prev = i
        prev_d0 = datetime.timedelta()
        prev_d1 = datetime.timedelta()
    t0 += [prev_d0 + datetime.timedelta(days=(np.clip(int(np.random.normal(155, 72)), 0, np.inf)))]
    t1 += [prev_d1 + datetime.timedelta(days=(np.clip(int(np.random.normal(147, 77)), 0, np.inf)))]

d = pd.Series([t1[i] if la_id[i] in target1 else t0[i] for i in range(n_a * n_l)])
loan['open_date'] = loan['application_date'] - d

In [13]:
loan['credit_type'] = [np.random.choice(['auto', 'mort', 'cc', 'pil', 'micro', 'business'], 1, p=[0.1, 0.1, 0.3, 0.4, 0.05, 0.05])[0] if la_id[i] in target1 else np.random.choice(['auto', 'mort', 'cc', 'pil', 'micro', 'business'], 1, p=[0.13, 0.05, 0.2, 0.5, 0.1, 0.02])[0] for i in range(n_a * n_l)]
loan['infosource'] = 'zub dayu'

In [14]:
loan['payment_string'] = '0'
aps = pd.Series([''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.71, 0.09, 0.05, 0.025, 0.025, 0.02, 0.015, 0.01, 0.005])[0] for j in range(int(min(np.random.uniform(12, 36), d[i].days / 30)))]) if la_id[i] in target1 else ''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.726, 0.088, 0.048, 0.023, 0.023, 0.018, 0.013, 0.008, 0.003])[0] for j in range(int(min(np.random.uniform(12, 36), d[i].days / 30)))]) for i in range(n_a * n_l)])
mps = pd.Series([''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.71, 0.09, 0.05, 0.025, 0.025, 0.02, 0.015, 0.01, 0.005])[0] for j in range(int(min(np.random.uniform(36, 360), d[i].days / 30)))]) if la_id[i] in target1 else ''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.742, 0.086, 0.046, 0.021, 0.021, 0.016, 0.011, 0.006, 0.001])[0] for j in range(int(min(np.random.uniform(36, 360), d[i].days / 30)))]) for i in range(n_a * n_l)])
cps = pd.Series([''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.71, 0.09, 0.05, 0.025, 0.025, 0.02, 0.015, 0.01, 0.005])[0] for j in range(int(min(np.random.uniform(1, 12), d[i].days / 30)))]) if la_id[i] in target1 else ''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.726, 0.088, 0.048, 0.023, 0.023, 0.018, 0.013, 0.008, 0.003])[0] for j in range(int(min(np.random.uniform(1, 12), d[i].days / 30)))]) for i in range(n_a * n_l)])
pps = pd.Series([''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.71, 0.09, 0.05, 0.025, 0.025, 0.02, 0.015, 0.01, 0.005])[0] for j in range(int(min(np.random.uniform(3, 12), d[i].days / 30)))]) if la_id[i] in target1 else ''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.726, 0.088, 0.048, 0.023, 0.023, 0.018, 0.013, 0.008, 0.003])[0] for j in range(int(min(np.random.uniform(3, 12), d[i].days / 30)))]) for i in range(n_a * n_l)])
mips = pd.Series([''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.71, 0.09, 0.05, 0.025, 0.025, 0.02, 0.015, 0.01, 0.005])[0] for j in range(int(min(np.random.uniform(1, 3), d[i].days / 30)))]) if la_id[i] in target1 else ''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.726, 0.088, 0.048, 0.023, 0.023, 0.018, 0.013, 0.008, 0.003])[0] for j in range(int(min(np.random.uniform(1, 3), d[i].days / 30)))]) for i in range(n_a * n_l)])
bps = pd.Series([''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.71, 0.09, 0.05, 0.025, 0.025, 0.02, 0.015, 0.01, 0.005])[0] for j in range(int(min(np.random.uniform(12, 360), d[i].days / 30)))]) if la_id[i] in target1 else ''.join([np.random.choice(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 1, p=[0.05, 0.726, 0.088, 0.048, 0.023, 0.023, 0.018, 0.013, 0.008, 0.003])[0] for j in range(int(min(np.random.uniform(12, 360), d[i].days / 30)))]) for i in range(n_a * n_l)])
loan['payment_string'][loan['credit_type'] == 'auto'] = aps[loan['credit_type'] == 'auto']
loan['payment_string'][loan['credit_type'] == 'mort'] = mps[loan['credit_type'] == 'mort']
loan['payment_string'][loan['credit_type'] == 'cc'] = cps[loan['credit_type'] == 'cc']
loan['payment_string'][loan['credit_type'] == 'pil'] = pps[loan['credit_type'] == 'pil']
loan['payment_string'][loan['credit_type'] == 'micro'] = mips[loan['credit_type'] == 'micro']
loan['payment_string'][loan['credit_type'] == 'business'] = bps[loan['credit_type'] == 'business']

In [15]:
pay = loan[['loan_id', 'payment_string']]
pay.columns = ['pay_loan_id', 'ps']
pay['m'] = ((loan['application_date'] - loan['open_date']) / 30).dt.days

In [16]:
newc = ['p' + str(i) for i in range(max([len(j) for j in pay['ps']]))]
pay['ps'] = pd.Series(['_'.join(i) for i in pay['ps']])
pay[newc] = pay['ps'].str.rsplit('_', expand=True)

In [17]:
pay = pd.wide_to_long(pay[['pay_loan_id', 'm'] + newc], stubnames='p', i=['pay_loan_id', 'm'], j='pos').reset_index().dropna().reset_index()

In [18]:
pay['months_ago'] = pay['m'] - pay['pos']
pay['index'] = [i for i in range(pay.shape[0])]

In [19]:
pay = pay[['index', 'pay_loan_id', 'months_ago', 'p']][pay['p'] != '']
pay.columns = ['payment_id', 'pay_loan_id', 'months_ago', 'payment_status']

In [20]:
appl.to_sql(name='application',
            con=engine, 
            schema='Modeling.dbo',
            if_exists='replace',
            index=False)

In [21]:
over.to_sql(name='BKI_overview',
            con=engine, 
            schema='Modeling.dbo',
            if_exists='replace',
            index=False)

In [22]:
loan.to_sql(name='BKI_loans',
            con=engine, 
            schema='Modeling.dbo',
            if_exists='replace',
            index=False, 
            chunksize=100000)

In [23]:
pay.to_sql(name='BKI_payments',
           con=engine,
           schema='Modeling.dbo',
           if_exists='replace',
           index=False, 
           chunksize=100000)